In [0]:
sc

Out[1]:

SparkContext 

 Spark UI 

 
 Version 
 v3.1.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# As recommended by book
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [0]:
# Setup paths and directories for this lab
from os.path import join
data_path = 'dbfs:/FileStore/tables/lab8'
static_file_path = join(data_path, 'static')
stream_file_path = join(data_path, 'stream')
parquet_file_path = join(data_path, 'lab8.parquet')
parquet_checkpoint_path = join(data_path, 'parquet_checkpoint')

## cleanup and make parquet checkpoint directory
dbutils.fs.rm(parquet_checkpoint_path, recurse=True)
dbutils.fs.mkdirs(parquet_checkpoint_path)
## cleanup and make parquet file apth directory
dbutils.fs.rm(parquet_file_path, recurse=True)
dbutils.fs.mkdirs(parquet_file_path)

# Create the directory that will hold your "live streaming files"
dbutils.fs.rm(f"{stream_file_path}", recurse=True)
dbutils.fs.mkdirs(f"{stream_file_path}")
dbutils.fs.ls(f"{stream_file_path}")

Out[3]: []

In [0]:
# supporting method used in this lab to copy files to stream path
def cp_file(file_name):
  dbutils.fs.cp(f"{static_file_path}/{file_name}", 
                f"{stream_file_path}/{file_name}")

In [0]:
# Verify that there are four log files in your staging directory
len(dbutils.fs.ls(static_file_path)) == 4

Out[5]: True

In [0]:
#  Create the schema for the log files
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

csv_to_read_schema = StructType([
  StructField("serverID",StringType(),False),
  StructField("severity",IntegerType(),False),
  StructField("timestamp",IntegerType(),False)])

In [0]:
# Create the streaming DataFrame (readStream) on your log directory, using the schema you just created
csv_read_stream = spark.readStream\
  .schema(csv_to_read_schema)\
  .option("maxFilesPerTrigger",1)\
  .csv(stream_file_path)

In [0]:
# Create the streaming DataFrame (readStream) on your log directory, using the schema you just created
rawData = csv_read_stream\
  .writeStream\
  .queryName("all_data")\
  .format("memory")\
  .outputMode("append")\
  .start()

In [0]:
# all data obtained from read stream
all_data = spark.sql("select * from all_data")

Out[9]: 0

In [0]:
# Use the data frame you just created to create another data frame with the 
# sev2 volume report.  It should have columns 'serverID' and 'avgVolume'

# Create and start a query (writeStream) that generates the sev2 report;  it is an in-memory sink.
sev2Report = csv_read_stream\
  .filter(csv_read_stream.severity==2)\
  .groupBy("serverID").count()\
  .withColumnRenamed("count", "avgVolume")\
  .writeStream\
  .queryName("sev2_group_by_server")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [0]:
# Write a (very simple) spark SQL query to show the contents of your query.  It should initially be empty
sev2_volume_report = spark.sql("SELECT * FROM sev2_group_by_server")
sev2_volume_report.show()

+--------+---------+
serverID|avgVolume|
+--------+---------+
+--------+---------+

In [0]:
# Copy two files into your 'live data' directory for both servers, for both servers and time period 1 through 5
cp_file('s115.csv')
cp_file('s215.csv')

In [0]:
# Rerun the query to show that the sev2 volume report has been updated
sev2_volume_report.show()

+--------+---------+
serverID|avgVolume|
+--------+---------+
 s2| 4600|
 s1| 1897|
+--------+---------+

In [0]:
# Now copy the log files for times 6 to 10
cp_file('s1610.csv')
cp_file('s2610.csv')

In [0]:
# Run the query again to verify that the report was updated. Be sure to wait for a little while
# to make sure the query is updated.
sev2_volume_report.show()

+--------+---------+
serverID|avgVolume|
+--------+---------+
 s2| 5199|
 s1| 1997|
+--------+---------+

In [0]:
# Delete all files from your "live" directory
dbutils.fs.rm(f"{stream_file_path}", recurse=True)
dbutils.fs.mkdirs(f"{stream_file_path}")
dbutils.fs.ls(f"{stream_file_path}")

Out[21]: []

In [0]:
# Create a data frame on top of your original data frame that holds the raw data, 
# this data frame for the sev0 report is just <serverID> <time stamp>
from pyspark.sql.functions import col
sev0Report = csv_read_stream\
  .filter(csv_read_stream.severity==0)\
  .select(col("serverID"), col("timestamp"))

In [0]:
# # # Create a query on your sev0 data frame that writes the table to a parquet file, 
# # #  appending new records to the filehttps://community.cloud.databricks.com/?o=5292400056450288#
# # # https://stackoverflow.com/questions/55859868/pyspark-structured-streaming-write-to-parquet-in-batches

#  Create the schema for the log files
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

ser0_parquet_schema = StructType([
  StructField("serverID",StringType(),False),
  StructField("timestamp",IntegerType(),False)])


from os.path import join
parquetFile = join(parquet_file_path, "output.parquet")
sev0parquetWrite = csv_read_stream\
  .filter(csv_read_stream.severity==0)\
  .select(col("serverID"), col("timestamp"))\
  .writeStream\
  .format("parquet")\
  .outputMode("append")\
  .option("path", parquet_file_path)\
  .trigger(processingTime='2 seconds')\
  .option("checkpointLocation", parquet_checkpoint_path)\
  .start()

In [0]:
dbutils.fs.ls(parquet_file_path)

Out[26]: [FileInfo(path='dbfs:/FileStore/tables/lab8/lab8.parquet/_spark_metadata/', name='_spark_metadata/', size=0)]

In [0]:
# Display the query content by reading the parquet file (it should be empty)
spark.read.schema(ser0_parquet_schema).parquet(parquet_file_path).show()

+--------+---------+
serverID|timestamp|
+--------+---------+
+--------+---------+

In [0]:
# Copy in the files for timestamp 1 through 5
cp_file('s115.csv')
cp_file('s215.csv')

In [0]:
# Display the query again by reading the parquet file.  Are there new records?
spark.read.parquet(parquet_file_path).show()

+--------+---------+
serverID|timestamp|
+--------+---------+
 s1| 5|
 s1| 5|
 s1| 5|
+--------+---------+

In [0]:
# Copy in the files for timestamp 6 through 10
cp_file('s1610.csv')
cp_file('s2610.csv')

In [0]:
# Display the query again by reading the parquet file.  Are there new records?
spark.read.parquet(parquet_file_path).show()

+--------+---------+
serverID|timestamp|
+--------+---------+
 s1| 5|
 s1| 5|
 s1| 5|
+--------+---------+

In [0]:
# Be tidy, stop all your streaming queries!
for s in spark.streams.active:
  s.stop()

In [0]:
# Verify that there are no active streams
len(spark.streams.active) == 0

Out[33]: True